### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git


### 安装特定依赖

In [ ]:
! cd modelscope-agent && pip install -r requirements.txt
! pip install transformers_stream_generator
! pip install invisible-watermark>=2.0

In [ ]:
import os
os.chdir('modelscope-agent/demo')


import sys
sys.path.append('../')


### 读取环境变量文件和工具config配置

In [ ]:
from dotenv import load_dotenv
import os
from modelscope.utils.config import Config

# 配置环境变量，里面需要配置modelscope token来remote调用AI模型
os.environ['TOOL_CONFIG_FILE'] ='../config/cfg_tool_template.json'
os.environ['MODEL_CONFIG_FILE'] ='../config/cfg_model_template.json'

os.environ['OUTPUT_FILE_DIRECTORY'] = './tmp'

# ModelScope Token从这里获取: https://modelscope.cn/my/myaccesstoken
print('请输入ModelScope Token，可以从这里获取: https://modelscope.cn/my/myaccesstoken')
os.environ['MODELSCOPE_API_TOKEN'] = input()

print('请输入OPENAI_API_KEY')

os.environ['OPENAI_API_KEY'] = input()


# 读取工具调用CONFIG文件，包括工具的名称和远程请求url
tool_cfg_file = os.getenv('TOOL_CONFIG_FILE')
print(tool_cfg_file)
tool_cfg = Config.from_file(tool_cfg_file)

model_cfg_file = os.getenv('MODEL_CONFIG_FILE')
print(model_cfg_file)
model_cfg = Config.from_file(model_cfg_file)
model_name = 'openai'

### 构建langchian tool

In [ ]:

from modelscope_agent.agent import AgentExecutor
from modelscope_agent.tools import LangchainTool
from langchain.tools import ShellTool


shell_tool = LangchainTool(ShellTool())


additional_tool_list = {
    shell_tool.name: shell_tool
}

### Agent构建

In [ ]:
from modelscope_agent.prompt import PromptGenerator
from modelscope_agent.llm import LLMFactory
CHATGPT_STSTEM_TEMPLATE = """<|system|>:: You are an assistant tries to help human solve problems with tools."""
CHATGPT_INSTRUCTION_TEMPLATE = """The following lists the tools available for the current session. 
The job of you to come up with a series of simple commands that will perform the task the human wants to perform. 
If you need to call the plug-in, you should give the commands in following format: 
\'<|startofthink|>```JSON{\"api_name\": {api_name},\
 \"parameters\": {\"parameter1\": \"value1\", \"parameter2\": \"value2\"}}```<|endofthink|>\'. 
After calling plug-in, you need to generate a reasonable sumarization based on the execution result. 
If you think there is no need to call the plug-in, you can directly give the corresponding reply. \n\n<tool_list>"""


CHATGPT_USER_TEMPLATE = """<|user|>:<user_input>"""

CHATGPT_EXEC_TEMPLATE = """<|startofexec|><exec_result><|endofexec|>\n"""

CHATGPT_ASSISTANT_TEMPLATE = """<|assistant|>:"""

prompt_generator = PromptGenerator(CHATGPT_STSTEM_TEMPLATE, CHATGPT_INSTRUCTION_TEMPLATE, CHATGPT_USER_TEMPLATE, CHATGPT_EXEC_TEMPLATE, CHATGPT_ASSISTANT_TEMPLATE)

llm = LLMFactory.build_llm(model_name, model_cfg)
agent = AgentExecutor(llm, tool_cfg, additional_tool_list=additional_tool_list, prompt_generator=prompt_generator)

## tool检验

In [ ]:
agent.reset()
agent.run('生成一张鲜花的图片')

In [ ]:
agent.reset()
agent.run('please execuute command ls')

In [ ]:
agent.reset()

# remote=True为调用modelscope api，该服务免费支持QPS较低，建议部署在本地，将remote=False
agent.run('使用地址识别模型，从下面的地址中找到省市区等元素，地址：浙江杭州市江干区九堡镇三村村一区')